## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-10-17-14-19 +0000,nyt,Operator of Eddie Bauer Stores Files for Bankr...,https://www.nytimes.com/2026/02/10/business/ed...
1,2026-02-10-17-13-57 +0000,bbc,Pakistan tune up for India with win against USA,https://www.bbc.com/sport/cricket/articles/cx2...
2,2026-02-10-17-13-46 +0000,startribune,Live: ICE director vows to release bodycam foo...,https://www.startribune.com/ice-raids-minnesot...
3,2026-02-10-17-13-30 +0000,nypost,Mark Zuckerberg latest tech billionaire to shu...,https://nypost.com/2026/02/10/us-news/mark-zuc...
4,2026-02-10-17-12-19 +0000,nyt,"Ahead of Jobs Report, White House Seeks to Dow...",https://www.nytimes.com/2026/02/10/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2421/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
61,trump,32
41,epstein,24
63,new,22
57,canada,14
14,ice,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
208,2026-02-10-01-54-00 +0000,wsj,Trump Threatens to Block Opening of New Bridge...,https://www.wsj.com/world/americas/trump-threa...,102
123,2026-02-10-11-32-07 +0000,nypost,Trump told Palm Beach police chief ‘thank good...,https://nypost.com/2026/02/10/us-news/trump-to...,96
198,2026-02-10-02-41-35 +0000,nyt,Trump Threatens to Block Opening of New Bridge...,https://www.nytimes.com/2026/02/09/us/politics...,96
159,2026-02-10-09-57-46 +0000,nypost,Trump says he will block US-Canada Bridge unle...,https://nypost.com/2026/02/10/us-news/trump-sa...,95
242,2026-02-09-23-15-50 +0000,cbc,Trump threatens to block opening of new bridge...,https://www.cbc.ca/news/canada/trump-gordie-ho...,90


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
208,102,2026-02-10-01-54-00 +0000,wsj,Trump Threatens to Block Opening of New Bridge...,https://www.wsj.com/world/americas/trump-threa...
117,72,2026-02-10-11-41-00 +0000,wsj,"The U.S. economy looks healthy, but it crossed...",https://www.wsj.com/finance/stocks/healthcare-...
123,64,2026-02-10-11-32-07 +0000,nypost,Trump told Palm Beach police chief ‘thank good...,https://nypost.com/2026/02/10/us-news/trump-to...
230,55,2026-02-10-00-06-00 +0000,wsj,Soaring profits and stocks funnel more of GDP ...,https://www.wsj.com/economy/jobs/capital-labor...
222,48,2026-02-10-01-08-36 +0000,nyt,"Savannah Guthrie Says She Believes Her Mom, Na...",https://www.nytimes.com/2026/02/09/us/savannah...
218,44,2026-02-10-01-13-00 +0000,wsj,The White House is at loggerheads with Congres...,https://www.wsj.com/politics/policy/proposed-b...
43,38,2026-02-10-15-46-00 +0000,wsj,Target to Lay Off 500 Workers as New CEO Shake...,https://www.wsj.com/business/retail/target-to-...
28,37,2026-02-10-16-15-01 +0000,nypost,Gavin Newsom ripped by LA’s top fed prosecuto...,https://nypost.com/2026/02/10/us-news/gavin-ne...
271,34,2026-02-09-21-55-13 +0000,nypost,Bad Bunny slammed for controversial Super Bowl...,https://nypost.com/2026/02/09/us-news/bad-bunn...
211,33,2026-02-10-01-44-00 +0000,wsj,Immigration Court Drops Case Against Tufts Stu...,https://www.wsj.com/us-news/law/immigration-co...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
